<a href="https://colab.research.google.com/github/hiya906/my-machine-learning/blob/master/Day_3_Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **사용법**

1.   우측 상단 '로그인'
2.   좌측 상단 '실습 모드에서 열기'


※ 각각의 셀은 셀 좌측 상단 실행 버튼을 통해 실행할 수 있습니다.

※ 실행 중 '경고: 이 노트는 Google에서 작성하지 않았습니다.'라는 창이 뜰 경우, '실행 전에 모든 런타임 재설정'란에 체크 후 '무시하고 계속하기'를 하시면 됩니다.

# Neural Machine Translation


## Package import

필요한 package 들을 import 시켜줍니다. 

Colab에서 제공하는 tensorflow는 버전이 1.14 입니다.
따라서, tensorflow package를 import 시키기 전에 tensorflow 2.0 버전을 pip install을 사용하여 설치해줍니다.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

import unicodedata
import re
import numpy as np
import os
import io
import time

## Prepare the dataset

https://github.com/jungyeul/korean-parallel-corpora 에서 제공하는 학습 데이트를 사용합니다.
아래 명령어로 data 폴더를 만든 후, 폴더 안에 train dataset 과 test 데이터 셋을 끌어 놓습니다.


In [0]:
# 이미 data 폴더가 생성되어 있다면 이 부분을 주석처리 해주세요
os.mkdir('./data')

train_path = 'data/train'
test_path = 'data/test' 

src = 'ko'
tgt = 'en'

### 1. 문장 내의 특수 문자, 혹은 변환할 수 없는 문자들을 제거하고, 문장의 양 끝에 각각 문장의 시작과 끝을 나타내는 토큰 삽입합니다.


In [0]:
def unicode_to_ascii(s, l):
    return ''.join(c for c in unicodedata.normalize(l, s)
        if unicodedata.category(c) != 'Mn')

def normalize(s, l='en'):
    cls = ''
    if l=='en':
        cls = 'NFD'
    elif l=='ko':
        cls = 'NFKC'
    s = unicode_to_ascii(s.lower().strip(), cls)
    s = re.sub(r'[" "]+', " ", s)
    if l == 'en':
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def preprocess_sentence(w, l):
    w = normalize(w,l)
    w = w.rstrip().strip()
    if l == 'en':
        w = '^ ' + w + ' $'
    elif l == 'ko':
        w = '^' + w + '$'
    return w

In [0]:
en_sentence = u"May I borrow this book?"
ko_sentence = u"제가 이 책을 빌려도 될까요?"
print(preprocess_sentence(en_sentence, 'en').encode('utf-8'))
print(preprocess_sentence(ko_sentence, 'ko').encode('utf-8'))

### 2. 같은 뜻을 가진 Source와 Target 문장이 각각의 dataset 내에서 같은 순서에 위치하도록 pairing 합니다.


In [0]:
def create_dataset(path, src, tgt, num_examples):
    src_lines = io.open( path + '.' + src, encoding='UTF-8').read().strip().split('\n')
    tgt_lines = io.open( path + '.' + tgt, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(s, src), preprocess_sentence(t, tgt)]  for s, t in zip(src_lines[:num_examples], tgt_lines[:num_examples])]

    return zip(*word_pairs)

In [0]:
en, ko = create_dataset(train_path, src, tgt, None)
print(en[-1])
print(ko[-1])

### 3. 언어의 특성에 맞게 문장을 토큰 단위로 분리하고 각각의 토큰을 고유한 숫자로 표기합니다.



In [0]:
def tokenize(lang, l):
    # 영어는 단어 단위, 한글은 음절 단위로 tokenizing 합니다.
    tok = False
    if l == 'en':
        tok = False
    elif l == 'ko':
        tok = True
    # tf.keras.preprocessing.text.Tokenizer 객체를 생성하고 각각의 언어에 fit 시켜줍니다.
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',char_level=tok)
    lang_tokenizer.fit_on_texts(lang)
    return lang_tokenizer
  
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      # tf.keras.preprocessing.text.Tokenizer의 기본 함수를 이용하여 쉽게 단어를 index로 변환할 수 있습니다.
      print ("%d ----> %s" % (t, lang.index_word[t]))
  return

def build_vocab(tr, te, l):
  lang_tokenizer = tokenize(tr.__add__(te), l)
  # 문장을 토큰 단위의 list로 변환합니다.
  tr_tensor = lang_tokenizer.texts_to_sequences(tr)
  te_tensor = lang_tokenizer.texts_to_sequences(te)
  # 가장 긴 문장 길이에 맞춰, 문장 뒤쪽으로 padding 해줍니다.
  tr_tensor = tf.keras.preprocessing.sequence.pad_sequences(tr_tensor, padding='post')
  te_tensor = tf.keras.preprocessing.sequence.pad_sequences(te_tensor, padding='post')
  return tr_tensor, te_tensor, lang_tokenizer

In [0]:
# 데이터 셋을 로드합니다.
num_examples = 30000
inp_tr_lang, tgt_tr_lang = create_dataset(train_path, src, tgt, num_examples)
inp_te_lang, tgt_te_lang = create_dataset(test_path, src, tgt, num_examples)

# 데이터를 tensor의 형태로 변환합니다.
inp_tr_tensor, inp_te_tensor, inp_lang = build_vocab(inp_tr_lang, inp_te_lang, src)
tgt_tr_tensor, tgt_te_tensor, tgt_lang = build_vocab(tgt_tr_lang, tgt_te_lang, tgt)

# 최대 문장 길이를 구하는 부분입니다. tensorflow의 내장함수를 사용하여 padding 하였기 때문에 이미 모든 문장의 길이가 최대 문장 길이와 동일합니다.
max_length_tgt = len(tgt_te_tensor[0]) if len(tgt_tr_tensor[0]) < len(tgt_te_tensor[0]) else len(tgt_tr_tensor[0])
max_length_inp = len(inp_te_tensor[0]) if len(inp_tr_tensor[0]) < len(inp_te_tensor[0]) else len(inp_tr_lang[0])

In [0]:
# 데이터 각각의 개수를 확인합니다.
print(len(inp_tr_tensor), len(tgt_tr_tensor), len(inp_te_tensor), len(tgt_te_tensor))

In [0]:
print("Input Language; index to word mapping")
convert(inp_lang, inp_tr_tensor[0])
print()
print("Target Language; index to word mapping")
convert(tgt_lang, tgt_tr_tensor[0])

## Create tf.data.Dataset

전처리한 데이터를 이용하여 Dataset 객체를 생성합니다.


In [0]:
BUFFER_SIZE = len(inp_tr_tensor)
BATCH_SIZE = 32
steps_per_epoch = len(inp_tr_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(tgt_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((inp_tr_tensor, tgt_tr_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

## Write the encoder model


![대체 텍스트](https://postfiles.pstatic.net/MjAxOTA3MjFfOTAg/MDAxNTYzNzEwMzMxODM3.vCTeOFxSzl1rgXoEb9YQ5DAAQbowhEE8WEj1AEEMwDIg.8NHvtAOIivO7r15ko3jt7trDRocpSp2vCHYRv-XICPcg.PNG.er2357/Encoder.png?type=w773)

위와 같은 모델 구조를 가진 Encoder를 정의합니다. 정의한 Encoder 모델을 생성하고 호출하여 반환되는 hidden state를 context vector로 사용합니다.


In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    
    # Embedding layer를 정의합니다. 
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    # RNN layer를 정의합니다.
    self.rnn =  tf.keras.layers.GRU(self.enc_units, return_sequences= True, return_state = True)
    

  def call(self, x, hidden):
    # embedding 함수를 호출하여 x를 embedding 합니다.
    x = self.embedding(x)
    
    # hidden을 초기 state로 하고 embedding 된 문장을 rnn에 넣어줍니다.
    output, state = self.rnn(x)
    
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

## Write the Decoder Model

![대체 텍스트](https://postfiles.pstatic.net/MjAxOTA3MjFfMTgg/MDAxNTYzNzEwNDI4ODYw.MJxm0wmIKe661tz6Jf_kOjA9T9Cs0QxC91bhGNk3tUQg.WseYOuD4fM-nvNDBn7i6p_NWrqw_jDmvK06yicqtMIwg.PNG.er2357/Decoder.png?type=w773)

위와 같은 모델 구조를 가진 Decoder를 정의합니다. 정의한 Decoder 모델을 생성하고 호출할 때 반환되는 값들은 vocabulary 내의 단어들 각각이 현재 step 에서 반환될 확률을 의미합니다.

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    
    # Embedding layer를 정의합니다.
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) 
    
    # RNN layer를 정의합니다.
    self.rnn =  tf.keras.layers.GRU(self.dec_units, return_sequences= True, return_state = True) 
    
    # Fully Connected Layer를 정의합니다.
    self.fc = tf.keras.layers.Dense(vocab_size, activation='softmax')


  def call(self, x, hidden):
    # embedding 함수를 호출하여 x를 embedding 합니다.
    x = self.embedding(x)
    
    # context vector와 input을 합쳐줍니다.
    x = tf.concat([tf.expand_dims(hidden, 1), x], axis=-1)

    # RNN layer에 x를 입력합니다.
    output, state =  self.rnn(x)
    
    output = tf.reshape(output, (-1, output.shape[2]))
    
    # RNN의 output을 FCN layer에 입력합니다.
    x = self.fc(output)

    return x, state

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

## Define the optimizer and the loss function

In [0]:
# Adam optimizer로 옵티마이저를 설정해줍니다.
optimizer = tf.keras.optimizers.Adam()

# SparseCategoricalCrossentropy loss로 손실함수를 설정합니다.
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  
  # 손실 함수를 호출합니다.
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training



In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    # encoder를 호출하여 context vector를 얻습니다. initial state는 전달받은 enc_hidden으로 설정합니다.
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    # context vector를 decoder의 초기 hidden state로 설정합니다.
    dec_hidden = enc_hidden

    # decoder의 첫번째 input으로 시작토큰인 '^'를 모든 batch에 대해 설정해줍니다.
    dec_input = tf.expand_dims([tgt_lang.word_index['^']] * BATCH_SIZE, 1)

    
    for t in range(1, targ.shape[1]):
      
      # decoder를 호출하여 현재 step에서 등장할 단어를 예측합니다. initial state는 dec_hidden으로 설정합니다. 
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  # tape 변수를 사용하여 gradient를 계산합니다.
  gradients = tape.gradient(loss, variables)

  # optimizer로 계산한 gradient를 적용합니다.
  # 
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    
    # train_step을 호출하여 batch 당 loss 를 구합니다.
    batch_loss = train_step(inp, targ, enc_hidden)
    
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

## Translate


In [0]:
def evaluate(sentence, l):
    attention_plot = np.zeros((max_length_tgt, max_length_inp))

    sentence = preprocess_sentence(sentence, l)
    
    inputs = [i[0] for i in inp_lang.texts_to_sequences(sentence)]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tgt_lang.word_index['^']], 0)

    for t in range(max_length_tgt):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += tgt_lang.index_word[predicted_id] + ' '

        if tgt_lang.index_word[predicted_id] == '$':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [0]:
def translate(sentence, l):
    result, sentence = evaluate(sentence,l)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
translate(u'도쿄의 니케이 지수는 9291.03으로 0.33 퍼센트 하락했다.', 'ko')

In [0]:
translate(u'기이한 현상이다”고 덧붙였다.', 'ko')